<a href="https://colab.research.google.com/github/VolkanSimsir/LLM-FineTune-Course/blob/main/Gemma_3_4b_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsloth kullanarak Gemma-3 modelini Fine Tune etme işlemi için Türkçe Kılavuz.

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo transformers
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Fastmodel.from_pretrained() parametre açıklamaları:


**model_name** : Yüklemek istediğiniz önceden eğitilmiş modelin adını veya yolunu belirtir.

**max_seq_length**:Modelin işleyebileceği maksimum girdi dizisi uzunluğunu belirler. Daha uzun diziler, daha fazla bellek gerektirir ve performansı etkileyebilir. Genellikle 512, 1024 veya 2048 gibi değerler kullanılır.yani model 2048 token uzunluğuna kadar olan girdileri işleyebilir.

**load_in_4bit** :(Bellek Optimizasyonu) Modeli 4-bit quantize ile yükleyip yüklemeyeceğinizi belirtir.

**load_in_8bit**: (Bellek Optimizasyonu) Modeli 8-bit nicemleme ile yükleyip yüklemeyeceğinizi belirtir. 8-bit nicemleme, 4-bit'e göre daha az bellek tasarrufu sağlar, ancak genellikle daha iyi doğruluk korur.

**full_finetuning**:(Eğitim) Modelin tam finetuning (tüm parametrelerin güncellenmesi) yapılıp yapılmayacağını belirtir. False olarak ayarlanmış, yani sadece LoRA gibi parametre-verimli finetuning yöntemleri kullanılacak.

**quantization_config**: Quantize ayarlarını özelleştirmek için bir yapılandırma nesnesi sağlar.


In [3]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-29 13:17:46 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

**FastModel.get_peft_model** fonksiyonu, büyük dil modellerinin (LLM'ler) parametre verimli ince ayarı (PEFT) için kullanılan bir yöntemdir. Bu yöntem, modelin tüm parametrelerini güncellemek yerine, yalnızca küçük bir alt kümesini ince ayar yaparak hesaplama ve depolama maliyetlerini önemli ölçüde azaltır.


**finetune_vision_layers** : parametre, modelin görsel (image) katmanlarının ince ayar (fine-tuning) yapılıp yapılmayacağını belirler.

False değeri: Görsel katmanlar dondurulur, yani ağırlıkları güncellenmez. Bu, görsel özelliklerin önceden eğitilmiş halinin korunmasını sağlar ve ince ayar sırasında sadece dil katmanlarının öğrenmesine odaklanılır.


**model:** İnce ayar yapılacak temel model.

**finetune_language_layers**: Dil katmanlarının ince ayar yapılıp yapılmayacağını belirler. True olarak ayarlandığında, dil katmanlarının ince ayarı yapılır.

**finetune_attention_modules:** Dikkat modüllerinin ince ayar yapılıp yapılmayacağını belirler. True olarak ayarlandığında, dikkat modüllerinin ince ayarı yapılır.

**finetune_mlp_modules**: Çok katmanlı algılayıcı (MLP) modüllerinin ince ayar yapılıp yapılmayacağını belirler. True olarak ayarlandığında, MLP modüllerinin ince ayarı yapılır.

**r:** LoRA (Low-Rank Adaptation) matrisinin rank değeri. Bu değer, eklenen parametrelerin boyutunu belirler.

**lora_alpha:** LoRA'nın ölçekleme faktörü. Daha yüksek değerler, LoRA katmanlarının çıktısının daha fazla ölçeklenmesine neden olur.


**lora_dropout**: LoRA katmanlarında kullanılacak dropout oranı. Aşırı uyumu önlemek için kullanılır.


**bias:** Bias terimlerinin nasıl ele alınacağını belirler. "none" olarak ayarlandığında, bias terimleri güncellenmez.


**random_state:** Rastgelelik için kullanılan başlangıç değeri. Sonuçların tekrarlanabilirliğini sağlar.

In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


# Data Prep

### Neden Chat Template Kullanırız?

 Model eğitim sırasında ham metin üzerinden eğitilse de, sohbet uygulamalarında mesajların belirli formatlarda (örneğin, kullanıcı, asistan, sistem mesajları gibi) ayrılması gerekmektedir. Chat template, bu mesajları sabit bir şablona sokarak modelin hangi mesajın kimin tarafından gönderildiğini anlamasını kolaylaştırır.

 Chat template sayesinde, modelin eğitiminde kullanılan özel tokenlar eklenir. Bu tokenlar, modelin mesajlar arasındaki farkı belirlemesine ve daha tutarlı çıktılar üretmesine yardımcı olur.

 Tokenizer'a chat template eklememizin sebebi, modelin eğitiminde kullanılan formatla eşleşmesini sağlamak ve doğru tokenizasyonu garanti etmektir.


**get_chat_template fonksiyonu**, belirli bir sohbet şablonuna uygun olarak bir tokenizer'ı yapılandırır. Bu, modelin çoklu sohbetlerde kullanıcı ve model mesajlarını doğru şekilde ayırt etmesini sağlar. "gemma-3" şablonu, Gemma 3 modelinin sohbet formatına uygun bir yapılandırma sağlar.

Bu yapılandırma, modelin sohbet sırasında kullanıcı ve model mesajlarını doğru bir biçimde işlemesine yardımcı olur.


**Unsloth çok sayıda Chat Template desteklemekte zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3 ve daha fazlasını.**



---

Gemma-3 için Multi Turn Conversation şu şekilde:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```






In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

### mlabonne/FineTome-100k Verisetini kullanılıyor.

In [6]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

## standardize_data_formats Neden Kullanılır?

 Eğitim verilerini modelin beklediği standart bir formata dönüştürerek, verilerin tutarlı ve işlenebilir olmasını sağlar. Bu, modelin eğitim sürecinde verileri doğru bir şekilde işlemesine ve daha iyi performans göstermesine yardımcı olur.

Bu fonksiyon, farklı formatlardaki konuşma veri kümelerini (ShareGPT, Alpaca, ChatML vb.), modelin eğitiminde kullanılabilir tek bir standart formata dönüştürmek için tasarlanmıştır.

**Problem:** Farklı kaynaklardan gelen veriler farklı yapılarda olabilir (örneğin, biri Alpaca diğeri ShareGPT formatında).

**Çözüm:** Tüm veri kümelerini modelin anlayabileceği ortak bir formatta birleştirir.

In [7]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [8]:
dataset[100]

{'conversations': [{'content': 'What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?',
   'role': 'user'},
  {'content': 'In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Theref

## Gemma-3 chat templati datasete uyguluyoruz.

konuşma (chat) veri kümesini dil modelinin anlayabileceği formata dönüştürmek için kullanılır. Özellikle tokenizer'ın chat template'ini kullanarak ham konuşma verilerini modele uygun tokenize edilmiş metne çevirir.

In [9]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

# Model eğitimi için parametreleri ayarlıyoruz.

TRL Kütüphanesi: Hugging Face'in Supervised Fine-Tuning (SFT) için optimize edilmiş bir kütüphanedir.

Hiperparametreler, modelin performansını ve eğitim süresini doğrudan etkiler. Bu ayarlar, donanımınıza ve görevinize göre özelleştirilmelidir.

---

**SFTConfig**, eğitim sürecini kontrol eden hiperparametreleri bir arada tutar. Hiperparametreler, modelin nasıl öğreneceğini belirleyen ayarlardır.


* **per_device_train_batch_size**:
Her GPU'da bir batch'te işlenecek örnek sayısı.

* **gradient_accumulation_steps**
Gradyanları biriktirerek efektif batch boyutunu artırır.

* **warmup_steps**
Learning rate'in başlangıçta yavaşça artması için adım sayısı.

* **max_steps = 30**:
Toplam eğitim adımı (epoch yerine).
Eğitimi 30 adımla sınırlar

* **learning_rate**
Hızlı öğrenme için yüksek, ancak overfitting'i önlemek için uzun eğitimlerde düşürülür.

* **logging_steps = 1**:
Her 1 adımda log kaydı yapar.


* **optim**
8-bit quantize edilmiş AdamW optimizasyonu (bellek tasarrufu).

* **weight_decay = 0.01**
Regularization için ağırlık azaltma katsayısı.
Ağırlıkların büyümesini sınırlayarak overfitting'i azaltır.

* **lr_scheduler_type	"linear"**
Learning rate'in lineer olarak azalması.





In [10]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [11]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><bos><start_of_turn>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<end_of_turn>\n<start_of_turn>model\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefo

### **train_on_responses_only Neden Kullanılır?**

Bu fonksiyon, modelin eğitim sırasında sadece asistan (model) yanıtları üzerinde loss hesaplamasını sağlar. Kullanıcı girdilerini (user kısımları) loss hesaplamasından çıkararak modelin yanıt üretme kalitesini artırır.

**Problem:** Geleneksel eğitimde model, kullanıcı mesajlarını da "tahmin etmeye" çalışır.

**Çözüm:** Loss maskesiyle sadece model yanıtları üzerinde eğitim yapılır.


**Avantajları**

Model, kullanıcı mesajlarını tahmin etmek yerine sadece kendi yanıtlarını optimize eder.

Gereksiz hesaplamalar atlanarak eğitim süresi ve bellek kullanımı azalır.

Model, yanıt kalitesine odaklanır.

---

Neden Bu Fonksiyonu Kullanmalıyız?


Model, bir sohbet asistanıysa, görevi kullanıcıya yanıt vermektir, kullanıcıyı taklit etmek değil.


Bu yöntem, özellikle instruction fine-tuning ve Multi turn conversation uygulamarda önemlidir.

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

*Memory hakkında bilgi*

In [15]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.457 GB of memory reserved.


## Eğitim yapıyoruz.

In [16]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.188700
2,1.658300
3,1.708200
4,1.350000
5,1.124000
6,1.436500
7,0.777900
8,1.143000
9,0.924200
10,0.816100


In [17]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

434.8915 seconds used for training.
7.25 minutes used for training.
Peak reserved memory = 6.23 GB.
Peak reserved memory for training = 0.773 GB.
Peak reserved memory % of max memory = 42.263 %.
Peak reserved memory for training % of max memory = 5.244 %.


# Inference
Inference (Çıkarım): Eğitilmiş bir modelin yeni girdilere dayanarak tahmin (bu durumda metin üretimi) yapması.


Gemma-3 ekibine göre inference için
uygun değerler temperature = 1.0, top_p = 0.95, top_k = 64.




**tokenizer = get_chat_template(tokenizer, chat_template = "gemma-3",)**

Ne Yapıyor: Daha önceden yüklenmiş olan tokenizer nesnesini alır ve get_chat_template fonksiyonu aracılığıyla ona "gemma-3" sohbet şablonunu uygular. Güncellenmiş tokenizer nesnesi tekrar aynı değişkene atanır.

Neden Yapılıyor: tokenizer'ın bundan sonraki işlemlerde (özellikle apply_chat_template metodunda) Gemma-3'ün beklediği doğru formatı kullanmasını sağlamak için. Bu, modelin girdiyi doğru yorumlaması için kritik öneme sahiptir.

**text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)**

Ne Yapıyor: Gemma-3 şablonu ile yapılandırılmış tokenizer'ı kullanarak messages listesini modelin işleyebileceği tek bir metin (string) haline getirir.

add_generation_prompt = True: Bu çok önemlidir. Şablonu uygularken, modelin cevap vermesi gerektiğini belirten özel token'ları veya metni (örn. <start_of_turn>model veya <|assistant|> gibi) otomatik olarak ekler.

Neden Yapılıyor: Ham messages yapısı model için doğrudan anlamlı değildir. Model, eğitildiği formata uygun, tek parça bir metin girdisi bekler. add_generation_prompt=True, modelin "şimdi senin sıran, cevap üret" komutunu almasını sağlar.

**outputs = model.generate(...)**

Ne Yapıyor: Modelin (model nesnesi) generate metodunu çağırarak asıl metin üretme işlemini başlatır.

**tokenizer(...) veya **inputs: Önceki adımda oluşturulan ve GPU'ya gönderilen token ID'lerini (input_ids) ve dikkat maskesini (attention_mask) içeren tensörleri generate fonksiyonuna argüman olarak iletir. ** operatörü, sözlükteki anahtar-değer çiftlerini otomatik olarak argüman adı=değer şeklinde fonksiyona geçirir.

**max_new_tokens = 64:** Modelin girdinin üzerine en fazla 64 yeni token üretmesini sınırlar. Daha uzun cevaplar için bu değer artırılabilir.

**temperature = 1.0:** Üretimdeki rastgeleliği kontrol eder. 1.0 görece yaratıcı ama bazen daha az tutarlı sonuçlar verebilir. Daha düşük değerler (örn. 0.7) daha odaklı ve deterministik, daha yüksek değerler daha rastgele çıktılar üretir.

**top_p = 0.95 (Nucleus Sampling):** Modelin bir sonraki token'ı seçerken olasılıkları kümülatif olarak %95'i oluşturan en olası token kümesinden seçim yapmasını sağlar. Bu, çok düşük olasılıklı token'ların seçilmesini engellerken çeşitliliği korur.

**top_k = 64:** Modelin bir sonraki token'ı seçerken sadece en olası 64 token arasından seçim yapmasını sağlar. top_p ile birlikte veya tek başına kullanılabilir.

**tokenizer.batch_decode(outputs)**

Ne Yapıyor: Modelin ürettiği outputs tensörünü (hem orijinal girdi token'larını hem de yeni üretilen token'ları içerir) alır ve bu sayısal token ID'lerini tekrar okunabilir metne dönüştürür. batch_decode, bir veya daha fazla diziyi (batch) işleyebilir.

Neden Yapılıyor: Modelin çıktısı sayısaldır. Sonucu görmek için bu sayıların tekrar metne çevrilmesi gerekir.

Kavram: Detokenization: Sayısal token ID'lerini metne dönüştürme.

In [18]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 7']

**TextStreamer**


Normalde **model.generate** fonksiyonu, tüm cevap üretilene kadar bekler ve sonra tüm metni tek seferde döndürür. Uzun cevaplar için bu, bir süre beklemeniz gerektiği anlamına gelir.

Cevabın tamamlanmasını beklemeden, modelin "düşünme" sürecini ve cevabın nasıl oluştuğunu gerçek zamanlı olarak izleyebilirsiniz. Bu, daha iyi bir kullanıcı deneyimi sunar.


In [19]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The sky is blue because of a phenomenon called Rayleigh scattering. This is when sunlight is scattered by the molecules in the atmosphere. Blue light is scattered more than other colors because it has a shorter wavelength. This is why we see a blue sky when we look up at the sky.<end_of_turn>


# Saving, loading finetuned models

**Aşağıdaki kodun mantığı**
model değişikliklerini (adaptörleri) ve bu süreçte kullanılan tokenizer'ı kaydeder.

**model.save_pretrained("gemma-3")**

Ne Yapar: Bu komut, fine-tuning sırasında eğitilen LoRA adaptörlerini yerel diskinizde "gemma-3" adlı bir klasöre kaydeder.

Neden Kullanılır: Fine-tuning sürecinde, orijinal büyük modelin (örn. Gemma-3) tüm ağırlıklarını değiştirmek yerine, sadece küçük "adaptör" katmanları ekledik ve eğittik (LoRA/QLoRA mantığı budur). Bu komut, sadece bu küçük, yeni eğitilmiş adaptör ağırlıklarını ve bunların nasıl uygulanacağını belirten bir yapılandırma dosyasını kaydeder.

**Model Şu Anda Ne Durumda?** model nesnesi şu anda bellekte hem orijinal (muhtemelen quantize edilmiş) Gemma-3 modelini hem de üzerine "takılmış" olan eğitilmiş LoRA adaptörlerini içerir. save_pretrained bu birleşik yapıdan sadece adaptör kısmını ayıklayıp kaydeder.



Bu komut, milyarlarca parametresi olan orijinal Gemma-3 modelinin tamamını KAYDETMEZ. Sadece üzerine eklediğiniz ve eğittiğiniz adaptörleri kaydeder. Bu yüzden çok hızlıdır ve çok az yer kaplar.

----
**tokenizer.save_pretrained("gemma-3")**

Ne Yapar: Fine-tuning sırasında kullanılan tokenizer yapılandırmasını (kelime dağarcığı, özel tokenlar, sohbet şablonu vb.) "gemma-3" klasörüne kaydeder.

Neden Kullanılır: Modeli daha sonra tekrar kullanmak istediğinizde, metni tam olarak aynı şekilde token'lara ayırmanız gerekir. Eğer tokenizer'da özel token'lar eklediyseniz veya get_chat_template ile bir sohbet şablonu ayarladıysanız, bu bilgilerin de kaydedilmesi gerekir. Bu komut, tutarlılığı sağlamak için tokenizer'ın durumunu kaydeder.

---

**model.push_to_hub(...) ve tokenizer.push_to_hub(...)**

Ne Yapar: save_pretrained ile aynı dosyaları (LoRA adaptörleri ve tokenizer dosyaları) yerel diske kaydetmek yerine doğrudan Hugging Face Hub'daki hesabınıza yükler.

Neden Kullanılır: Model adaptörlerinizi ve tokenizer'ınızı başkalarıyla kolayca paylaşmak, farklı ortamlardan erişmek veya sadece bulutta yedeklemek için kullanılır.

**HF'ye Ne Atıyoruz? ** Yerelde "gemma-3" klasörüne kaydedilecek olan dosyaların aynısını (adaptör ağırlıkları, adaptör config, tokenizer dosyaları) Hugging Face Hub'daki belirttiğiniz repo'ya (HF_ACCOUNT/gemma-3) atarsınız. Yine orijinal büyük model atılmaz, sadece küçük adaptörler ve tokenizer gider.

Unsloth ve QLoRA ile fine-tuning yaptığınızda, devasa ana modeli değil, sadece küçük LoRA adaptörlerini eğitirsiniz.

save_pretrained veya push_to_hub komutları, bu küçük adaptörleri ve tutarlılık için tokenizer yapılandırmasını kaydeder/yükler.

Bu yöntem, depolama alanı ve yükleme/indirme süreleri açısından çok verimlidir, çünkü gigabaytlarca veri yerine sadece megabaytlarca veri kaydedilir/paylaşılır.

Kaydedilen adaptörler, daha sonra orijinal temel modelin üzerine "takılarak" fine-tune edilmiş davranış elde edilir.

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained( # Kaydedilmiş LoRA adaptörlerini temel model üzerine yüklemek ve kullanıma hazır hale getirmek.
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

**model.save_pretrained_merged("gemma-3-finetune", tokenizer)**

Nedir? Bu komut, Unsloth'un bellekte tuttuğu modeli (yani orijinal temel model + üzerine uygulanmış LoRA adaptörleri) tek bir model halinde birleştirir ve bu birleştirilmiş modeli float16 (FP16) veri tipinde yerel diske kaydeder.

 Modeli başka bir yerde kullanmak istediğinizde, sadece bu birleştirilmiş modeli yüklemeniz yeterli olur; temel modeli ve adaptörleri ayrı ayrı yükleyip birleştirme adımıyla uğraşmazsınız.

 FP16, orijinal FP32'ye göre model boyutunu yarı yarıya azaltır ve genellikle performansı (hızı) artırır, doğrulukta ise çok az bir kayıp yaşanır (veya hiç yaşanmaz). vLLM gibi araçlar genellikle FP16 modellerle iyi çalışır.

 GGUF Öncesi Adım: Modeli GGUF gibi quantize edilmiş formatlara dönüştürmeden önce genellikle onu birleştirilmiş (merged) ve genellikle FP16 formatında kaydetmek yaygın bir adımdır.

save_pretrained sadece küçük LoRA adaptörlerini kaydeder (Megabayt boyutunda).

save_pretrained_merged ise temel modeli ve adaptörleri birleştirip ortaya çıkan tam modeli kaydeder (Gigabayt boyutunda, temel modele yakın).

**model.push_to_hub_merged(...)**

Nedir? Bu komut, save_pretrained_merged ile aynı birleştirme işlemini yapar (temel model + adaptörler), ancak sonucu yerel diske kaydetmek yerine doğrudan belirttiğiniz Hugging Face Hub reposuna yükler. Model yine genellikle FP16 formatında yüklenir.

Neden Kullanılır?

Paylaşım ve Erişim: Birleştirilmiş, kullanıma hazır modelinizi kolayca başkalarıyla paylaşmak veya farklı makinelerden/ortamlardan kolayca erişmek için kullanılır.

push_to_hub sadece küçük LoRA adaptörlerini Hugging Face'e yükler.

push_to_hub_merged tam, birleştirilmiş modeli Hugging Face'e yükler (GB boyutunda veri transferi).

**save_pretrained_merged,push_to_hub_merged **Her ikisi de sadece adaptörleri değil, tam modeli ele alır ve genellikle modeli farklı ortamlarda (VLLM, standart Transformers, GGUF dönüşümü vb.) daha kolay kullanılabilir hale getirmek için yapılır.

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

**GGUF**

GGUF, model ağırlıklarını daha verimli bir şekilde depolayarak, daha hızlı yükleme ve çıkarım (inference) sağlar.

llama.cpp Entegrasyonu: GGUF, llama.cpp ile doğrudan uyumludur, bu da CPU üzerinde LLM'leri çalıştırmayı kolaylaştırır.

CPU Üzerinde LLM Çalıştırma: GGUF, LLM'leri GPU olmadan CPU üzerinde çalıştırmak için idealdir.

Düşük Kaynaklı Cihazlarda LLM Kullanma: GGUF, düşük bellek ve işlem gücüne sahip cihazlarda LLM'leri çalıştırmayı mümkün kılar.

Model Paylaşımı: GGUF, LLM'leri kolayca paylaşmak ve dağıtmak için standart bir format sağlar.

Quantization : GGUF'un en büyük avantajlarından biri, modelleri farklı seviyelerde quantize etme yeteneğidir. Bu, modelin ağırlıklarını daha düşük hassasiyetli veri tiplerine (örn. 8-bit, 4-bit tamsayılar) dönüştürerek dosya boyutunu önemli ölçüde küçültür ve özellikle CPU üzerinde inference (çıkarım) hızını artırır. Q8_0, Q4_K_M gibi farklı quantization türleri, boyut/hız ve kalite arasında farklı dengeler sunar.

Neden GGUF? Modeli standart Python/PyTorch ortamı dışında, daha hafif araçlarla (LM Studio, Ollama, llama.cpp komut satırı aracı vb.) çalıştırmak, CPU üzerinde performanslı inference yapmak veya modeli daha kolay dağıtmak için kullanılır.

 Unsloth ile ince ayar yaptıktan sonra elde edeceğiniz model, genellikle orijinal formatında (örneğin, safetensors) olur. Ancak, bu formatı doğrudan Ollama gibi araçlarla kullanmak zor olabilir. GGUF formatı, özellikle CPU üzerinde daha verimli çalışmak için tasarlanmıştır ve Ollama ile uyumludur. Bu nedenle, ince ayarlı modelinizi GGUF'a dönüştürmek, Ollama'da kullanabilmeniz için gereklidir.


**llama.cpp**
llama.cpp, LLM'leri CPU üzerinde çalıştırmak için yüksek performans sağlar.

Düşük Kaynak Tüketimi: llama.cpp, düşük bellek ve işlem gücü gerektirir.

Python Entegrasyonu: llama.cpp-python, llama.cpp'nin yeteneklerini Python ortamında kullanmanızı sağlar.

CPU Üzerinde LLM Çalıştırma: llama.cpp-python, LLM'leri CPU üzerinde çalıştırmak için kolay bir yol sağlar.

LLM'leri Python Uygulamalarına Entegre Etme: llama.cpp-python, LLM'leri Python uygulamalarına entegre etmeyi kolaylaştırır.


In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )